# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance==0.2.38
!pip install backtesting
!pip install bokeh==2.4.3 # 繪圖套件
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.52
    Uninstalling yfinance-0.2.52:
      Successfully uninstalled yfinance-0.2.52
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.3
    Uninstalling bokeh-3.6.3:
      Successfully uninstalled bokeh-3.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
holoviews 1.20.0 requires bokeh>=3.1, but you have bokeh 2.4.3 which is incompatible.
panel 1.6.0 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 2.4.3 which is incompatible.


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [2]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ma1,ma2
Date,,,,,,,,
2020-02-14,337.0,337.0,334.5,335.0,301.655396,16933683,NaN,NaN
2020-02-17,331.5,333.0,330.5,331.5,298.503815,15937079,NaN,NaN
2020-02-18,324.5,326.5,322.0,322.0,289.949432,61825604,NaN,NaN
2020-02-19,322.5,327.0,322.0,326.5,294.001434,38781218,NaN,NaN
2020-02-20,328.0,329.0,325.0,325.5,293.100983,27011736,328.1,NaN


### 3️⃣ 定義回測策略

In [3]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [4]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    85.03289
Equity Final [$]                     99221.53
Equity Peak [$]                     100468.84
Commissions [$]                        314.97
Return [%]                           -0.77847
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                    -0.16183
Volatility (Ann.) [%]                 0.29006
CAGR [%]                             -0.10774
Sharpe Ratio                         -0.55791
Sortino Ratio                        -0.77523
Calmar Ratio                         -0.12926
Max. Drawdown [%]                      -1.252
Avg. Drawdown [%]                    -0.10575
Max. Drawdown Duration      843 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                                   64
Win Rate [%]                          42.1875
Best Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag
0,1,29,50,284.0,299.0,None,None,15.0,0.052817,2020-03-27,2020-04-29,33 days,None
1,1,53,57,296.5,300.0,None,None,3.5,0.011804,2020-05-05,2020-05-11,6 days,None
2,1,62,63,291.0,294.0,None,None,3.0,0.010309,2020-05-18,2020-05-19,1 days,None
3,1,69,71,297.0,292.0,None,None,-5.0,-0.016835,2020-05-27,2020-05-29,2 days,None
4,1,72,85,294.0,314.5,None,None,20.5,0.069728,2020-06-01,2020-06-18,17 days,None


### 5️⃣ 回測繪圖

In [5]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

Row(id='1407', ...)

### 6️⃣ 設定停利、停損點

In [6]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    52.54934
Equity Final [$]                     99601.45
Equity Peak [$]                    100032.468
Commissions [$]                        336.15
Return [%]                           -0.39855
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                    -0.08273
Volatility (Ann.) [%]                 0.14358
CAGR [%]                             -0.05507
Sharpe Ratio                         -0.57617
Sortino Ratio                        -0.75357
Calmar Ratio                         -0.19199
Max. Drawdown [%]                    -0.43088
Avg. Drawdown [%]                    -0.06129
Max. Drawdown Duration     1639 days 00:00:00
Avg. Drawdown Duration      222 days 00:00:00
# Trades                                   70
Win Rate [%]                         38.57143
Best Trade [%]                    

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [7]:
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### 8️⃣ 創建 Gemini-2.0-Flash 模型函式

In [8]:
# Gemini 2.0 Flash 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(
        model="gemini-2.0-flash",
        n=1,
        messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content."
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self)."

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [9]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
#user_msg = ["RSI", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
code_str=code_str.replace('```','')
code_str=code_str.replace('python','')
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()

[*********************100%%**********************]  1 of 1 completed



import pandas as pd

def calculate(df):
    """
    Calculates the MACD (Moving Average Convergence Divergence) for a given dataframe.

    Args:
        df (pd.DataFrame): A dataframe containing OHLCV data with columns
                           'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'.

    Returns:
        pd.DataFrame: The input dataframe with added MACD-related columns:
                      'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'MACD_Histogram'.
    """

    # Calculate 12-day EMA of the 'Adj Close' price
    df['EMA_12'] = df['Adj Close'].ewm(span=12, adjust=False).mean()

    # Calculate 26-day EMA of the 'Adj Close' price
    df['EMA_26'] = df['Adj Close'].ewm(span=26, adjust=False).mean()

    # Calculate MACD line
    df['MACD'] = df['EMA_12'] - df['EMA_26']

    # Calculate signal line (9-day EMA of MACD)
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate MACD Histogram
    df['MACD_Histogram'] = df['MACD'] - df['Signal_L

,Open,High,Low,Close,Adj Close,Volume,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram
Date,,,,,,,,,,,
2025-02-10,1125.0,1125.0,1095.0,1105.0,1105.0,28527108,1106.725631,1097.061642,9.663989,10.424980,-0.760991
2025-02-11,1110.0,1115.0,1100.0,1110.0,1110.0,18898928,1107.229380,1098.020039,9.209341,10.181852,-0.972511
2025-02-12,1110.0,1115.0,1100.0,1100.0,1100.0,24172954,1106.117168,1098.166703,7.950465,9.735575,-1.785110
2025-02-13,1090.0,1095.0,1080.0,1090.0,1090.0,33210403,1103.637604,1097.561762,6.075842,9.003628,-2.927786
2025-02-14,1065.0,1070.0,1060.0,1060.0,1060.0,64486940,1096.924126,1094.779409,2.144717,7.631846,-5.487129


### 🔟 策略生成

In [10]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
strategy_str=strategy_str.replace('```','')
strategy_str=strategy_str.replace('python','')
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)


class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data.MACD, self.data.Signal_Line):
            self.buy(size=1,
                     sl=self.data.Close[-1] * 0.90,
                     tp=self.data.Close[-1] * 1.20)
        elif crossover(self.data.Signal_Line, self.data.MACD):
            self.sell(size=1,
                      sl=self.data.Close[-1] * 1.10,
                      tp=self.data.Close[-1] * 0.80)

-----------------------
Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    93.58553
Equity Final [$]                    99740.648
Equity Peak [$]                    100210.488
Commissions [$]                       470.072
Return [%]                           -0.25935
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                     -0.0538
Volatility (Ann.) [%]        

### 1️⃣1️⃣ 寫成函式

In [11]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  df = yf.download(stock_id, period=period)

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  code_str=code_str.replace('```','')
  code_str=code_str.replace('python','')
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  strategy_str=strategy_str.replace('```','')
  strategy_str=strategy_str.replace('python','')
  print(strategy_str)

  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [12]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [13]:
stats = ai_backtest(stock_id="2330.TW",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


[*********************100%%**********************]  1 of 1 completed



class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data.MACD, self.data['Signal Line']):
            self.buy(size=1,
                     sl=self.data.Close[-1] * 0.90,
                     tp=self.data.Close[-1] * 1.20)
        elif crossover(self.data['Signal Line'], self.data.MACD):
            self.sell(size=1,
                      sl=self.data.Close[-1] * 1.10,
                      tp=self.data.Close[-1] * 0.80)

-----------------------
Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    93.91447
Equity Final [$]                   99708.3572
Equity Peak [$]                   100231.0572
Commissions [$]                      484.9228
Return [%]                           -0.29164
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                    -0.06051
Volatility (Ann.) [%]  

### 1️⃣4️⃣ 比較多個策略

In [14]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")
# 策略2:SMA
stats2 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="SMA",
            add_msg="無")
# 策略3:RSI+停利停損
#stats3 = ai_backtest(stock_id="2330.TW", period="5y",
#            user_msg="RSI",
#            add_msg="請設置10%的停損點與20%的停利點")

#reply = backtest_analysis(stats1,stats2,stats3)
reply = backtest_analysis(stats1,stats2)
print(reply)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data['MACD'], self.data['Signal Line']):
            self.buy(size=1,
                     sl=self.data.Close[-1] * 0.9,
                     tp=self.data.Close[-1] * 1.2)
        elif crossover(self.data['Signal Line'], self.data['MACD']):
            self.sell(size=1,
                      sl=self.data.Close[-1] * 1.1,
                      tp=self.data.Close[-1] * 0.8)

-----------------------
Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    93.91447
Equity Final [$]                   99708.3572
Equity Peak [$]                   100231.0572
Commissions [$]                      484.9228
Return [%]                           -0.29164
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                    -0.06051
Volatility (Ann.) [%]


class AiStrategy(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(self.data.Close, self.data.SMA):
            self.buy()
        elif crossover(self.data.SMA, self.data.Close):
            self.sell()


-----------------------
Start                     2020-02-14 00:00:00
End                       2025-02-14 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    97.03947
Equity Final [$]                    58621.234
Equity Peak [$]                     150984.22
Commissions [$]                     98985.866
Return [%]                          -41.37877
Buy & Hold Return [%]               216.41791
Return (Ann.) [%]                   -10.47745
Volatility (Ann.) [%]                23.97175
CAGR [%]                             -7.10174
Sharpe Ratio                         -0.43707
Sortino Ratio                        -0.59508
Calmar Ratio                         -0.16848
Max. Drawdown [%]                 